# 深入理解协程-coroutine

从语法上来看，协程和生成器类似，都是定义体中包含yield关键字的函数。但是又不一样,协程重点在流程控制方面

### yield在协程中的用法

* 在协程中yield通常出现在表达式的右边，例如：data = yield,可以产出值，
* 也可以不产出--如果yield关键字后面没有表达式，那么生成器产出None.
* 协程可能从调用方接受数据，调用方是通过send(datum)的方式把数据提供给协程使用，而不是next(...)函数，通常调用方会把值推送给协程。
* 协程可以把控制权让给中心调度程序，从而激活其他的协程

所以总体上在协程中把yield看做是**控制流程的方式**。

### 协程运行过程

In [8]:
def simple_coroutine():
    print('---> coroutine started')
    x = yield
    print('---> coroutine received:', x)
    
my_coro = simple_coroutine()
print('定义一个生成器')
print(my_coro)
print('启动激活生成器')
print(next(my_coro))
print('向生成器传递值')
print(my_coro.send(10))

定义一个生成器
<generator object simple_coroutine at 0x000002A80A6D6048>
启动激活生成器
---> coroutine started
None
向生成器传递值
---> coroutine received: 10


StopIteration: 

对上述例子的分析：
* yield 的右边没有表达式，所以这里默认产出的值是None
* 刚开始先调用了next(...)是因为这个时候生成器还没有启动，没有停在yield那里，这个时候也是无法通过send发送数据。所以当我们通过next(...)激活协程后，程序就会运行到x = yield，这里有个问题我们需要注意，x = yield这个表达式的计算过程是先计算等号右边的内容，然后在进行赋值，所以当激活生成器后，程序会停在yield这里，但并没有给x赋值。
* 当我们调用send方法后yield会收到这个值并赋值给x,而当程序运行到协程定义体的末尾时和用生成器的时候一样会抛出StopIteration异常

如果协程没有通过next(...)激活(同样我们可以通过send(None)的方式激活)，但是我们直接send，会提示如下错误：

In [10]:
def simple_coroutine():
    print('---> coroutine started')
    x = yield
    print('---> coroutine received:', x)
    
my_coro = simple_coroutine()
print('定义一个生成器')

my_coro.send(12)

定义一个生成器


TypeError: can't send non-None value to a just-started generator

关于调用next(...)函数这一步通常称为”预激(prime)“协程，即让协程向前执行到第一个yield表达式，准备好作为活跃的协程使用

### 协程状态

协程在运行过程中有四个状态：

1. GEN_CREATE:等待开始执行
2. GEN_RUNNING:解释器正在执行，这个状态一般看不到
3. GEN_SUSPENDED:在yield表达式处暂停
4. GEN_CLOSED:执行结束

In [24]:
from inspect import getgeneratorstate

def simple_coro(a):
    print('-->started: a=', a)
    b = yield a
    print('-->recevied: b=', b)
    c = yield a+b
    print('-->recevied: c=', c)
    
coro = simple_coro(10)

print(getgeneratorstate(coro))
print(next(coro))
print(getgeneratorstate(coro))
print(coro.send(20))

try:
    coro.send(30)
except StopIteration as e:
    print(e)
    print(e.value)
    
print(getgeneratorstate(coro))

GEN_CREATED
-->started: a= 10
10
GEN_SUSPENDED
-->recevied: b= 20
30
-->recevied: c= 30

None
GEN_CLOSED


想要开始使用协程的时候必须通过next(...)方式激活协程，如果不预激，这个协程就无法使用，

如果哪天在代码中遗忘了那么就出问题了，所以有一种预激协程的装饰器，预激装饰器的演示例子::

In [22]:
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

@coroutine
def averager():
    total = 0.0
    count= 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count
  
coro_avg = averager()
from inspect import getgeneratorstate
print(getgeneratorstate(coro_avg))
print(coro_avg.send(10))
print(coro_avg.send(20))
print(coro_avg.send(30))


GEN_SUSPENDED
10.0
15.0
20.0


关于预激，在使用yield from句法调用协程的时候，会自动预激活，这样其实与我们上面定义的coroutine装饰器是不兼容的，在python3.4里面的asyncio.coroutine装饰器不会预激协程，因此兼容yield from

### 中止协程和异常处理

协程中为处理的异常会向上冒泡,传给next函数或send函数的调用方(即触发协程的对象)

如果我们send一个错误的数据就会报错，并且这个时候协程就被终止了,协程状态是GEN_CLOSED.

从python2.5开始客户端代码在生成器对象上调用两个方法: 分别为：throw和close，显示的把异常发送给协程

generator.throw:会让生成器在暂停的yield表达式处抛出指定的异常，
* 如果生成器处理了抛出的异常，代码会向前执行到下一个yield表达式，而产出的值会成为调用generator.throw方法代码的返回值。
* 如果生成器没有处理抛出的异常，异常会向上冒泡，传到调用方的上下文中。

generator.close:会让生成器在暂停的yield表达式处抛出GeneratorExit异常。
* 如果生成器没有处理这个异常，或者抛出了StopIteration异常，调用方不会报错
* 如果收到GeneratorExit异常，生成器一定不能产出值，否则解释器会抛出RuntimeError异常。生成器抛出的异常会向上冒泡，传给调用方。

In [26]:
class DemoException(Exception):
    """定义异常类型"""
    pass

def demo_exc_handling():
    print('--> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('DemoException handled. continuing...')
        else:
            print('-->coroutiine received:{!r}'.format(x))
    raise RuntimeError('this line should never run.')
 

exc_coro = demo_exc_handling()

next(exc_coro)

exc_coro.send(10)

exc_coro.send(20)

exc_coro.send(30)

exc_coro.throw(TypeError)

from inspect import getgeneratorstate

getgeneratorstate(exc_coro)

--> coroutine started
-->coroutiine received:10
-->coroutiine received:20
-->coroutiine received:30


TypeError: 

当传入我们定义的异常时不会影响协程，协程不会停止，可以继续send,但是如果是没有处理的异常的时候，就会报错，并且协程会被终止

In [ ]:
'''gevent对协程的支持,本质上是greenlet在实现切换工作,'''

# gevent使用流程
# spawn方法可以看作是创建一个协程, joinall方法可以看作是添加任务,并启动运行协程
import requests
import gevent
from gevent import monkey
monkey.patch_all()


def run_task(url):
    try:
        r = requests.get(url)
    except Exception as e:
        print(e)

if __name__ == '__main__':
    urls = ['url1', 'url2', 'url3']
    greenlets = [gevent.spawn(run_task, url) for url in urls]
    gevent.joinall(greenlets)


# 协程池的使用
# -*- coding:utf-8 -*-
import gevent
from gevent import monkey
from gevent.pool import Pool
monkey.patch_all()
exist_pages = []
with open("xxxxx.txt", "r", encoding="utf-8") as f:
    for line in f:
        exist_pages.append(int(line.strip()))


def get_page_index():
    for page in range(1, 623000):
        yield page


def record_leak_pages(page):
    with open("xxxx.txt", "a", encoding="utf-8") as f:
        f.write("{}\n".format(page))


def check_page_exist(page):
    if page not in exist_pages:
        record_leak_pages(page)


pool = Pool(200)
try:
    pool.map(check_page_exist, get_page_index())
except Exception as e:
    print(e)


# asycio使用
# asyncio是Python 3.4版本引入的标准库，直接内置了对异步IO的支持。
# asyncio的编程模型就是一个消息循环。我们从asyncio模块中直接获取一个EventLoop的引用，
# 然后把需要执行的协程扔到EventLoop中执行，就实现了异步IO。
# asycio执行一个任务
import asyncio

@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    r = yield from asyncio.sleep(1)
    print("Hello again!")

# 获取EventLoop:
loop = asyncio.get_event_loop()
# 执行coroutine
loop.run_until_complete(hello())
loop.close()

# 使用3.5版本async和await关键字
import asyncio
async def hello():
    print('hello,world')
    r=await asyncio.sleep(1)
    print('hello,again')

loop=asyncio.get_event_loop()
loop.run_until_complete(hello())
loop.close()

# asycio执行多个任务
import threading
import asyncio

@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    yield from asyncio.sleep(1)
    print('Hello again! (%s)' % threading.currentThread())

loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

# 使用asycio异步网络连接来获取sina、sohu和163的网站首页,并打印header内容

import asyncio

@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()